# 영화 평점 분석 실습

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 1. 영화 평점 데이터 적재 및 전처리

In [ ]:
# 사용자 데이터 읽어오기
users = pd.read_csv('data/movielens/users.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '성별','연령','직업','지역'])
users.head()

In [ ]:
# 평점 데이터 읽어오기
ratings = pd.read_csv('data/movielens/ratings.dat', sep = '::', engine = 'python',
                   names = ['사용자아이디', '영화아이디','평점','타임스탬프'])
ratings.head()

In [ ]:
# 영화데이터 읽어오기
movies = pd.read_csv('data/movielens/movies.dat', sep = '::', engine = 'python',
                   names = ['영화아이디','영화제목','장르'], encoding = 'latin-1')
movies.head()

In [ ]:
# 3개의 데이터프레임을 하나로 합치기
data = pd.merge(users, ratings)
data = pd.merge(data, movies)
data.head()

In [ ]:
print('사용자 수:', len(users))
print('리뷰 수:', len(ratings)) #ratings.사용자아이디.count()
print('영화 수:', len(movies))

In [ ]:
# 모든 사용자가 리뷰를 했나?
ratings.사용자아이디.nunique() #모든 사용자가 영화 리뷰에 참여!

In [ ]:
# 리뷰가 없는 영화는 있는가?
ratings.nunique() #영화 수는 3883, 영화 아이디는 3706으로, 176개는 영화 리뷰가 없음!

## 2. 보고 싶은 영화 찾기
영화들의 평점 평균을 구하여, 사람들에게 인정받는 (평점이 높은) 영화 찾기

In [ ]:
data.head()

In [ ]:
# 영화들의 평점 평균을 구하여, 평점이 높은 영화 찾기 
data.pivot_table(index='영화제목', aggfunc='mean', values='평점')\
                 .sort_values(by='평점', ascending=False).head(10)

In [ ]:
data.pivot_table(index='영화제목', aggfunc='mean', values='평점').nlargest(10, '평점') #nsmallest

평균 평점이 만점인 영화들이 최상위에 위치함. 
일반적으로 평점이 만점인 경우는 대부분 평점의 개수가 매우 적은 경우이므로, 이를 확인하기 위해 평점의 개수도 함께 구해본다. 

In [ ]:
#영화제목도 중복될 수 있으므로, 제대로 하려면 영화아이디로 해야 됨
# 중복된 영화제목이 있는지 확인

movies.영화제목.nunique() #위에서 구한 영화 수와 일치하므로 중복X

In [ ]:
movies.nunique()

In [ ]:
#data.pivot_table(index=['영화아이디','영화제목'], aggfunc='mean', values='평점')\ 
#                 .nlargest(10, '평점') #nsmallest

In [ ]:
data.pivot_table(index='영화제목', aggfunc=['mean','count'], values='평점')

In [ ]:
data.pivot_table(index='영화제목', aggfunc=['mean','count'], values='평점')\
.nlargest(10, ('mean','평점')) #평점 높고, 개수도 많은 걸 찾자!

In [ ]:
#평점 평균이 4.5 이상이고, 평점의 개수가 1000개 이상인 영화를 보고싶은 영화로 지정!
영화평점 = data.pivot_table(index='영화제목', aggfunc=['mean','count'], values='평점')
영화평점.columns=['평점평균','평점개수']

In [ ]:
영화평점

In [ ]:
영화평점[(영화평점.평점평균 >= 4.5) & (영화평점.평점개수 >=1000)]

## [실습 #1] 여자들이 좋아하는 영화 찾기 
### - 여성 평점이 4.0 이상이고 여성 평점의 개수가 500개 이상인 영화

In [ ]:
data_f = data[data['성별']=='F']
data_f

In [ ]:
여성평점 = data_f.pivot_table(index='영화제목', aggfunc=['mean','count'], values='평점')
여성평점.columns=['평점평균','평점개수']

In [ ]:
여성평점[(여성평점.평점평균 >= 4.0) & (여성평점.평점개수 >=500)]

In [ ]:
data[data['성별']=='F'].pivot_table(index='영화제목', 
                                  aggfunc=['mean','count'], values='평점')

In [ ]:
ex1 = data.pivot_table(index='영화제목', columns='성별',                       
                       aggfunc=['mean','count'], values='평점')
ex1

In [ ]:
ex1[(ex1[('mean','F')]>= 4.0) & (ex1[('count','F')] >=500)]

## [실습 #2] 실습 #1에서 구한 영화(여성인기영화)의 장르를 분석해 보자.
여성인기영화의 장르 통계 구하기

예를 들어, 여성인기영화 중 Drama 장르의 영화는 10개, Action 영화는 3개, ...

In [ ]:
여성인기영화 = 여성평점[(여성평점.평점평균 >= 4.0) & (여성평점.평점개수 >=500)]
여성인기영화

In [ ]:
여성인기영화.index

In [ ]:
movies.영화제목.values

In [ ]:
lst=[]
for i in 여성인기영화.index:
    if i in movies.영화제목.values:
        lst.append(i)
lst

In [ ]:
여성인기영화장르 = movies[movies.영화제목.isin(lst)]
여성인기영화장르

In [ ]:
k =[]
for i in 여성인기영화장르.장르:
    for j in i.split('|'):
        k.append(j)

In [ ]:
k

In [ ]:
count={}
for i in k:
    try: count[i] += 1
    except: count[i]=1
print(count)

In [ ]:
ex2 = ex1[(ex1[('mean','F')]>= 4.0) & (ex1[('count','F')] >=500)].index

In [ ]:
ex2

In [ ]:
ex2_df = movies[movies.영화제목.isin(ex2)]
ex2_df

In [ ]:
ex2_df.장르.str.split('|')

In [ ]:
ex2_df = ex2_df.장르.str.split('|', expand=True)
ex2_df

In [ ]:
장르1 = ex2_df[0].value_counts()
장르2 = ex2_df[1].value_counts()
장르3 = ex2_df[2].value_counts()
장르4 = ex2_df[3].value_counts()
장르5 = ex2_df[4].value_counts()

In [ ]:
#장르1+장르2+장르3+장르4+장르5
장르1.add(장르2, fill_value=0).add(장르3, fill_value=0).add(장르4, fill_value=0).add(장르5, fill_value=0)

In [ ]:
sr = Series()
for col in ex2_df.columns:
    sr = sr.add(ex2_df[col].value_counts(), fill_value=0)
sr

In [ ]:
sr.sort_values(ascending=False)

## [실습 #3] 남자와 여자의 호불호가 크게 갈리는 영화 10개 찾기
전체 평점의 개수가 500개 이상인 영화만 대상으로 함.

In [ ]:
data_f = data[data['성별']=='F']
data_f

In [ ]:
여성평점 = data_f.pivot_table(index='영화제목', aggfunc=['mean','count'], values='평점')
여성평점.columns=['평점평균_f','평점개수_f']
여성평점 = 여성평점[여성평점.평점개수_f >=500]
여성평점

In [ ]:
data_m = data[data['성별']=='M']
data_m

In [ ]:
남성평점 = data_m.pivot_table(index='영화제목', aggfunc=['mean','count'], values='평점')
남성평점.columns=['평점평균_m','평점개수_m']
남성평점 = 남성평점[남성평점.평점개수_m >=500]
남성평점

In [167]:
여남평점 = pd.concat(여성평점, 남성평점)
여남평점['평점차이'] = abs(여남평점['평점평균_f']-여남평점['평점평균_m'])
여남평점.nlargest(10, ('평점차이'))

,평점평균_f,평점개수_f,평점평균_m,평점개수_m,평점차이
영화제목,,,,,
Groundhog Day (1993),3.735562,658.0,4.041358,1620,0.305796
Saving Private Ryan (1998),4.114783,575.0,4.398941,2078,0.284159
Braveheart (1995),4.016484,546.0,4.297839,1897,0.281355
Pulp Fiction (1994),4.071956,542.0,4.346839,1629,0.274883
Star Wars: Episode V - The Empire Strikes Back (1980),4.106481,648.0,4.344577,2342,0.238096
Jurassic Park (1993),3.579407,573.0,3.814197,2099,0.234791
"Matrix, The (1999)",4.128405,514.0,4.362235,2076,0.233830
Star Wars: Episode VI - Return of the Jedi (1983),3.865237,653.0,4.069058,2230,0.203821
Star Wars: Episode IV - A New Hope (1977),4.302937,647.0,4.495307,2344,0.192371


In [ ]:
ex3 = data.pivot_table(index = '영화제목', columns= '성별',
                      values = '평점', aggfunc = ['mean', 'count'])

In [ ]:
ex3 = ex3[ex3[('count','F')]+ex3[('count','M')]>=500]
ex3

In [ ]:
ex3[ex3['count'].sum(axis=1) >= 500]

In [165]:
abs(ex3[('mean','F')] - ex3[('mean','M')]).nlargest(10)

영화제목
Dirty Dancing (1987)                      0.830782
Good, The Bad and The Ugly, The (1966)    0.726351
Dumb & Dumber (1994)                      0.638608
Evil Dead II (Dead By Dawn) (1987)        0.611985
Grease (1978)                             0.608224
Caddyshack (1980)                         0.573602
Animal House (1978)                       0.538286
Exorcist, The (1973)                      0.529605
Rocky Horror Picture Show, The (1975)     0.512885
Big Trouble in Little China (1986)        0.497078
dtype: float64

## [실습 #4] 연령대 별로 영화 평점 분석하기
연령대(10대 미만, 10대, 20대, ...50대) 컬럼을 추가한 후, 영화별 연령대별 영화평점 구하기

In [103]:
data

,사용자아이디,성별,연령,직업,지역,영화아이디,평점,타임스탬프,영화제목,장르
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,5949,M,18,17,47901,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,M,35,14,30030,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,M,18,17,92886,2845,1,958153068,White Boys (1999),Drama
1000207,5851,F,18,20,55410,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


In [104]:
max(data.연령)

56

In [ ]:
def ages(i):
    if i<10:
        return '10대 미만'
    elif i<20:
        return '10대'
    elif i<30:
        return '20대'
    elif i<40:
        return '30대'
    elif i<50:
        return '40대'
    else:
        return '50대'

In [ ]:
연령평점 = data.pivot_table(index=['영화제목'], columns='연령대' 
                        aggfunc=['mean'], values='평점')
연령평점.columns=['평점평균']

In [ ]:
#data.pivot_table(index=['영화아이디','영화제목'], aggfunc='mean', values='평점')\ 
#                 .nlargest(10, '평점') #nsmallest

In [168]:
def generate_ages(x):
    if x<10:
        return '10대 미만'
    elif x<20:
        return '10대'
    elif x<30:
        return '20대'
    elif x<40:
        return '30대'
    elif x<50:
        return '40대'
    elif x<60:
        return '50대 이상'

In [169]:
generate_ages(56)

'50대 이상'

In [ ]:
data['연령대'] = data.연령.apply(generate_ages)

In [ ]:
data

In [ ]:
연령평점 = data.pivot_table(index='영화제목', columns='연령대', 
                        aggfunc=['mean'], values='평점')